In [19]:
%load_ext line_profiler

In [27]:
%load_ext snakeviz

The snakeviz extension is already loaded. To reload it, use:
  %reload_ext snakeviz


In [24]:
@profile
def read_grid(filename):
    grid = []
    with open(filename) as f:
        w, h = map(int, f.readline().split(maxsplit=1))
        
        for y in range(h):
            grid.append([0] * w)
        
        for no, line in enumerate(f):
            try:
                y, x = map(int, line.split(maxsplit=1))

                if y < 0 or x < 0:
                    raise ValueError

            except ValueError:
                raise Exception(f"Invalid cell on line {no + 2}.")

            grid[y][x] = 1

    return grid

Rules:
- Any live cell with fewer than two live neighbours dies, as if by underpopulation.
- Any live cell with two or three live neighbours lives on to the next generation.
- Any live cell with more than three live neighbours dies, as if by overpopulation.
- Any dead cell with exactly three live neighbours becomes a live cell, as if by reproduction.

In [28]:
@profile
def tick(grid):
    h, w = len(grid), len(grid[0])

    nextgrid = []
    for y in range(h):
        nextgrid.append([0] * w)

    for y, row in enumerate(grid):
        for x, cell in enumerate(row):
            count = 0
            if y > 0:
                count += grid[y-1][x-1] if x > 0 else 0
                count += grid[y-1][x]
                count += grid[y-1][x+1] if x < w - 1 else 0
            count += grid[y][x-1] if x > 0 else 0
            count += grid[y][x+1] if x < w - 1 else 0
            if y < h - 1:
                count += grid[y+1][x-1] if x > 0  else 0
                count += grid[y+1][x] 
                count += grid[y+1][x+1] if x < w - 1 else 0

            nextgrid[y][x] = 1 if count == 3 or (cell and count == 2) else 0

    return nextgrid

In [29]:
filename = "data/input_5x5.txt"

In [30]:
%lprun -f read_grid read_grid(filename)
grid = read_grid(filename)

Timer unit: 1e-07 s

Total time: 0.0013667 s
File: C:\Users\jazib\AppData\Local\Temp\ipykernel_27588\531961763.py
Function: read_grid at line 2

Line #      Hits         Time  Per Hit   % Time  Line Contents
     2                                           def read_grid(filename):
     3         1         14.0     14.0      0.1      grid = []
     4         2      11307.0   5653.5     82.7      with open(filename) as f:
     5         1       1164.0   1164.0      8.5          w, h = map(int, f.readline().split(maxsplit=1))
     6                                                   
     7         6         90.0     15.0      0.7          for y in range(h):
     8         5        112.0     22.4      0.8              grid.append([0] * w)
     9                                                   
    10         6        531.0     88.5      3.9          for no, line in enumerate(f):
    11         5         54.0     10.8      0.4              try:
    12         5        199.0     39.8      

In [31]:
grid

[[0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0],
 [0, 0, 0, 1, 0],
 [0, 1, 1, 1, 0],
 [0, 0, 0, 0, 0]]

In [32]:
%lprun -f tick tick (grid)
nextgrid = tick(grid)

Timer unit: 1e-07 s

Total time: 0.0004489 s
File: C:\Users\jazib\AppData\Local\Temp\ipykernel_27588\3829533973.py
Function: tick at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           @profile
     2                                           def tick(grid):
     3         1         30.0     30.0      0.7      h, w = len(grid), len(grid[0])
     4                                           
     5         1         12.0     12.0      0.3      nextgrid = []
     6         6         84.0     14.0      1.9      for y in range(h):
     7         5        123.0     24.6      2.7          nextgrid.append([0] * w)
     8                                           
     9         6         99.0     16.5      2.2      for y, row in enumerate(grid):
    10        30        453.0     15.1     10.1          for x, cell in enumerate(row):
    11        25        231.0      9.2      5.1              count = 0
    12        25        2

In [16]:
nextgrid

[[0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 1, 0, 1, 0],
 [0, 0, 1, 1, 0],
 [0, 0, 1, 0, 0]]

In [17]:
num_generations = 4

nextgrid = grid.copy()
for i in range(num_generations):
    nextgrid = tick(nextgrid)

nextgrid

[[0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 1, 0],
 [0, 0, 0, 0, 1],
 [0, 0, 1, 1, 1]]